Вначале используем DecisionTreeClassifier

In [3]:
#обычное решающее дерево
import pandas
import numpy as np
from sklearn.tree import DecisionTreeClassifier
#from sklearn import tree
data = pandas.read_csv('C:\\RLib\\titanic.csv', index_col='PassengerId')
data_sel = data.loc[:, data.columns.isin(['Survived', 'Pclass', 'Fare', 'Age', 'Sex'])]
data_sel = data_sel.dropna()
data_sel['Sex'] = np.where(data_sel['Sex'] == 'male', 0, 1)
Survived = data_sel.loc[:, data_sel.columns.isin(['Survived'])]
X = data_sel.loc[:, data_sel.columns.isin(['Pclass', 'Fare', 'Age', 'Sex'])]


from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(X, Survived, test_size=.33, random_state=1)


T = DecisionTreeClassifier(random_state=241, max_depth = 4)

T = T.fit(x_train, y_train)
T

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=241, splitter='best')

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
#обычное решающее дерево, автоматический подбор удачных параметров дерева

grid_search_cv = GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       iid='warn', n_jobs=None,
       param_grid={'max_depth': list(range(2, 30)), 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)
grid_search_cv.fit(x_train, y_train)

grid_search_cv.best_estimator_
#наилучшая высота дерева - 7

TypeError: DecisionTreeClassifier.__init__() got an unexpected keyword argument 'min_impurity_split'

In [9]:
#обычное решающее дерево: с ростом максимальной высоты дерева когда-то точность перестаёт расти - мы получили дерево, которое уже не меняется
#начиная с некоторой высоты, точность падает - слишком много уровней близко по смыслу к переобучению
for i in range(5, 30):
    T = DecisionTreeClassifier(random_state=241, max_depth = i)
    T = T.fit(x_train, y_train)
    print(str(i) + ": " + str(T.score(x_validation, y_validation)))

5: 0.8305084745762712
6: 0.8305084745762712
7: 0.826271186440678
8: 0.8305084745762712
9: 0.8050847457627118
10: 0.8135593220338984
11: 0.7966101694915254
12: 0.7669491525423728
13: 0.7796610169491526
14: 0.7796610169491526
15: 0.7796610169491526
16: 0.7754237288135594
17: 0.7754237288135594
18: 0.7754237288135594
19: 0.7754237288135594
20: 0.7754237288135594
21: 0.7754237288135594
22: 0.7754237288135594
23: 0.7754237288135594
24: 0.7754237288135594
25: 0.7754237288135594
26: 0.7754237288135594
27: 0.7754237288135594
28: 0.7754237288135594
29: 0.7754237288135594


In [13]:
from sklearn.model_selection import cross_val_score
print("accuracy:"+str(np.average(cross_val_score(grid_search_cv.best_estimator_, x_validation, y_validation, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(grid_search_cv.best_estimator_, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(grid_search_cv.best_estimator_, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(grid_search_cv.best_estimator_, x_validation, y_validation, scoring='recall'))))

accuracy:0.7329330304013849
f1:0.6445033893309754
precision:0.6587301587301587
recall:0.6413793103448276


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, Fut

In [4]:
#случайный лес
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV

param_grid = { 'n_estimators': [200, 300, 400],'max_features': ['auto'],'max_depth' : list(range(1, 20)), 'criterion' :['gini']}

RFC = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv= 5, refit = True) 
RFC.fit(x_train, y_train)

print("accuracy:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(RFC(grid_search_cv.best_estimator_, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(RFC(grid_search_cv.best_estimator_, x_validation, y_validation, scoring='recall'))))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y wa

NameError: name 'cross_val_score' is not defined

In [6]:
from sklearn.model_selection import cross_val_score
print("accuracy:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='recall'))))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected

accuracy:0.7710699989181001


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected

f1:0.686692521830165


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected

precision:0.7229907773386034


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected

recall:0.6636015325670498


In [16]:
print("accuracy:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='recall'))))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected

accuracy:0.7879476360489018


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


f1:0.6912152566443498


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected

precision:0.7587050630528891


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


recall:0.6524904214559387


1